In [1]:
"""Sets the notebook to the project root"""

import sys, os
PROJECT_NAME = "analysis"
current_directory = os.getcwd().split("/")
while current_directory[-1] != PROJECT_NAME:
    current_directory = current_directory[:-1]
if len(current_directory) > 1:
    project_root = "/".join(current_directory)
    os.chdir(project_root)
else:
    raise Exception("ERROR: Project root not found")

from dotenv import load_dotenv
load_dotenv(dotenv_path="./env/env.env")

True

In [6]:
"""Initializes the database connection"""

import os
from bson import ObjectId
from src.classes.mongo_client import initialize

# Get MongoDB connection strings from environment variables
connection_strings = {
    "stg": os.getenv("MONGO_STAGE_URI"),
    "prd": os.getenv("MONGO_PROD_URI")
}

# Initialize database connections
dbs = initialize(connection_strings)



Database connections initialized successfully!
Staging DB: MongoClient(host=['landmarksid-staging-shard-00-02-me4vi.mongodb.net:27017', 'landmarksid-staging-shard-00-01-me4vi.mongodb.net:27017', 'landmarksid-staging-shard-00-00-me4vi.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin', replicaset='landmarksid-staging-shard-0', tls=True)
Production DB: MongoClient(host=['production-shard-00-01.gw4dh.mongodb.net:27017', 'production-shard-00-02.gw4dh.mongodb.net:27017', 'production-shard-00-00.gw4dh.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, authsource='admin', replicaset='atlas-4v0y1q-shard-0', tls=True)


In [7]:


sitecards = list(dbs["stg"]["poinetwork"]["sitecards"].find({"metadata.media_owner":"ONETProposal"}))

pois = []
for sitecard in sitecards:
    if len(sitecard["poi_candidates"]) > 0:
        pois.append(dbs["stg"]["poinetwork"]["pois"].find_one(
            {"_id": ObjectId(sitecard["poi_candidates"][0]["poiId"])}
        ))

print(len(pois))


111


In [10]:

from src.classes.organizations import Organizations

organizations = Organizations()

clientNames = [
    "oOh Media", 
    "MediaWorks", 
    "JCDecaux", 
    "LUMO", 
    "VAST Billboards", 
    "Go Media - sales", 
    "Ad-Vantage Media", # ONET
    "Media5" # ONET
]

analysis = {}
for client in clientNames:
    analysis[client] = {}
    for o in [o for o in organizations.supportedOrganizations["MediaOwners"] if o["archived"] == False]:
        if o["clientName"] == client:
            analysis[client][o["organizationName"]] = {
                "matchedPois": [],
                "totalPois": 0
            }


for sitecard in list(dbs["stg"]["poinetwork"]["sitecards"].find()):
    if len(sitecard["poi_candidates"]) > 0:
        poi = dbs["stg"]["poinetwork"]["pois"].find_one(
            {"_id": ObjectId(sitecard["poi_candidates"][0]["poiId"])}
        )

        for client in analysis:
            if poi["organizationName"] in analysis[client]:
                analysis[client][poi["organizationName"]]["matchedPois"].append(poi["_id"])

for client in analysis:
    for orgName in analysis[client]:
        analysis[client][orgName]["totalPois"] = len(list(dbs["stg"]["poinetwork"]["pois"].find({"organizationName": orgName})))
    
for client in analysis:
    for orgName in analysis[client]:
        print(f"{orgName} Total Pois: {analysis[client][orgName]["totalPois"]}, Matched Pois: {len(analysis[client][orgName]["matchedPois"])}, {round(len(analysis[client][orgName]["matchedPois"]) / analysis[client][orgName]["totalPois"] * 100, 2)}%")

oOhMedia - Roadside - Classic Total Pois: 3016, Matched Pois: 0, 0.0%
oOhMedia - Roadside - Digital Total Pois: 1998, Matched Pois: 0, 0.0%
oOhMedia - ShoppingCentre - Classic Total Pois: 285, Matched Pois: 0, 0.0%
oOhMedia - ShoppingCentre - Digital Total Pois: 512, Matched Pois: 0, 0.0%
MediaWorks - Airport - Classic Total Pois: 27, Matched Pois: 27, 100.0%
MediaWorks - Airport - Digital Total Pois: 7, Matched Pois: 4, 57.14%
MediaWorks - Roadside - Classic Total Pois: 227, Matched Pois: 219, 96.48%
MediaWorks - Roadside - Digital Total Pois: 90, Matched Pois: 84, 93.33%
JCDecaux - Airport - Digital Total Pois: 131, Matched Pois: 127, 96.95%
JCDecaux - Airport - Classic Total Pois: 57, Matched Pois: 55, 96.49%
JCDecaux - Roadside - Classic Total Pois: 248, Matched Pois: 207, 83.47%
JCDecaux - Roadside - Digital Total Pois: 84, Matched Pois: 81, 96.43%
LUMO - Roadside - Digital Total Pois: 79, Matched Pois: 54, 68.35%
VAST Billboards - Roadside - Digital Total Pois: 72, Matched Pois: 

In [14]:
sitecards = list(dbs["stg"]["poinetwork"]["sitecards"].find())

candidates = []
for sitecard in sitecards:
    if len(sitecard["poi_candidates"]) > 0:
        candidates.extend([ObjectId(c["poiId"]) for c in sitecard["poi_candidates"] if c["poiId"] is not None])

pois = list(dbs["stg"]["poinetwork"]["pois"].find(
    {
        "_id": {"$nin": candidates},
        "organizationName": {"$in": ["MediaWorks - Roadside - Classic"]}
    },
))

print(len(pois))

for poi in pois:
    print(poi["organizationName"], poi["_id"])
    break

20
MediaWorks - Roadside - Classic 66d30042dfac96b384daa0cf


In [12]:
candidates

['5eaa3d8c6d8faa000496f0f3',
 '63ce07958d35050004556a1d',
 '642a2ba1a307dd000e403f20',
 '5e686c46bce3d200049f4850',
 '5e686c48bce3d200049f4851',
 '5e686c49bce3d200049f4852',
 '5eb36e43d9bfbe000445cca2',
 '62c77ce97a19470010547b1c',
 '5e686c23bce3d200049f4841',
 '61de7473d06aeb0011a703a5',
 '5e686c24bce3d200049f4842',
 '673ff84d0cad72001223fa13',
 '5e686c26bce3d200049f4843',
 '629d5acc6ee0d900114095d7',
 '5e686c42bce3d200049f484d',
 '5e686c43bce3d200049f484e',
 '62c77ce173aae60011123653',
 '66948af26691d90012f65f69',
 '5eb36e35a0851f00044e0014',
 '61de7494bed1450012b32646',
 '5f0e6798ae7a260004ce752b',
 '68085b731793bd3987d577be',
 '5e686c2dfdf71c00040d296b',
 '5eb36e14a0851f00044e0002',
 '652626ecfb8c6800126e6c53',
 '5e686c31bce3d200049f4846',
 '5f0e679c5b12d10004ab8ffb',
 '615d93e7a861a40004a35b14',
 '5e686c33bce3d200049f4847',
 '5e686c2efdf71c00040d296c',
 '64546aea55d368000e25a830',
 '642a2ba0a307dd000e403f1a',
 '5e686c3fbce3d200049f484b',
 '63ce07958d35050004556a20',
 '5e686c2ffdf7